In [ ]:
import random
from Hourbid import *
from Fibid import *
from blockbid import *
import matplotlib.pyplot as plt    
from tqdm import tqdm
import copy

In [ ]:
from line_profiler import LineProfiler
from functools import wraps
 
%load_ext line_profiler

In [ ]:
BS_bids = Blockbid()[0]
BD_bids = Blockbid()[1]
HS_bids = HourBid()[0]
HD_bids = HourBid()[1]
FS_bids = Fibid()[0]
FD_bids = Fibid()[1]

In [ ]:
x = HS_bids['price'][0:1000]
y= HS_bids['Quantity'][0:1000]
x2 = HD_bids['price'][0:1000]
y2= HD_bids['Quantity'][0:1000]
plt.plot(x,y)
plt.plot(x2,y2)
plt.axvline(147)

In [ ]:
HD_bids

In [ ]:
HS_bids['Quantity'][0]-HD_bids['Quantity'][1000]

In [ ]:
def Pairs(postion,Hour,DF,QP):
    
    if QP == 'DQ':
        DF=DF.iloc[HIndex_D[Hour][0]:HIndex_D[Hour][1]+1]
        return [DF['Quantity'][postion+HIndex_D[Hour][0]],DF['Quantity'][postion+1+HIndex_D[Hour][0]]]
    if QP == 'SQ':
        DF=DF.iloc[HIndex_S[Hour][0]:HIndex_S[Hour][1]+1]
        return [DF['Quantity'][postion+HIndex_S[Hour][0]],DF['Quantity'][postion+1+HIndex_S[Hour][0]]]
    if QP == 'DP':
        DF=DF.iloc[HIndex_D[Hour][0]:HIndex_D[Hour][1]+1]
        return [DF['price'][postion+HIndex_D[Hour][0]],DF['price'][postion+1+HIndex_D[Hour][0]]]
    else:
        DF=DF.iloc[HIndex_S[Hour][0]:HIndex_S[Hour][1]+1]
        return [DF['price'][postion+HIndex_S[Hour][0]],DF['price'][postion+1+HIndex_S[Hour][0]]]
    
Delta_S = np.zeros([len(BS_bids),24])


Delta_D = np.zeros([len(BD_bids),24])
BD_bids  = Blockbid()[1]
for i in range (len(BD_bids)):
    if BD_bids['Hour'][i]+BD_bids['Duration'][i]<=25:
        Delta_D[i,BD_bids['Hour'][i]-1:(BD_bids['Hour'][i]+BD_bids['Duration'][i])-1] = 1

BS_bids = Blockbid()[0]
for i in range (len(BS_bids)):
    if BS_bids['Hour'][i]+BS_bids['Duration'][i]<=25:
        Delta_S[i,BS_bids['Hour'][i]-1:(BS_bids['Hour'][i]+BS_bids['Duration'][i])-1] = 1

In [ ]:
x=np.zeros(10)
x[1]=1
list(x)

In [ ]:
# Heuristic Algorithm for flexible bids

    
def HA_FB():
    temp1 = []
    for i in range(len(FS_bids)):
        temp1.append(random.randint(0,23))
        
    temp2 = []
    for i in range(len(FD_bids)):
        temp2.append(random.randint(0,23))
    res1 = []
    res2 = []
    for h in range(24):
        t1=np.zeros(len(FS_bids))
        t2=np.zeros(len(FD_bids))
        x1=np.where(np.array(temp1)==h)[0]
        x2=np.where(np.array(temp2)==h)[0]
        for j in x1:
            t1[j]=1
        res1.append(list(t1))
        for k in x2:
            t2[k]=1
        res2.append(list(t2))
    return res1,res2

In [ ]:
NoneChild_D=[]
Child_D = []
Parent_D=[]
NoneChild_S=[]
Child_S=[]
Parent_S=[]
for i in range(len(BD_bids)):
    if BD_bids.isnull()['LinkID'][i] == True:
        NoneChild_D.append(i)
    else:
        Parent_D.append(np.where(BD_bids['ID']==BD_bids['LinkID'][i])[0][0])
        Child_D.append(i)
for i in range(len(BS_bids)):
    if BS_bids.isnull()['LinkID'][i] == True:
        NoneChild_S.append(i)
    else:
        Parent_S.append(np.where(BS_bids['ID']==BS_bids['LinkID'][i])[0][0])
        Child_S.append(i)

In [ ]:
# Heuristic Algorithm for Block bids


def HA_B():

    BD=[ 0 for i in range(len(BD_bids))]
    BS=[ 0 for i in range(len(BS_bids))]
    for i in NoneChild_D:
        BD[i] = random.randint(0,1)
    for i in NoneChild_S:
        BS[i] = random.randint(0,1)
    return BS,BD

In [ ]:
def find_f(a1,a2,b1,b2,x):
    grad = (b2-b1)/(a2-a1)
    b = b2-grad*a2
    return grad*x+b

def aggregate(h,BS,BD,FS,FD):
    
    a = 0+h*1000
    b = 1000+h*1000
    agg_q=[]
    agg_p = []
    y1=list(reversed(HD_bids['Quantity'][a:b].tolist()))
    y2=HS_bids['Quantity'][a:b].tolist()
    x1 = list(reversed(HD_bids['price'][a:b].tolist()))
    x2 = HS_bids['price'][a:b].tolist()
    
    # add Block
    for i in range(len(BS)):
        if BS[i] * Delta_S[i][h] >0:
            k = BS_bids['Quantity'][i]
            y2 = list(map(lambda x:x+k,y2))
            # for j in range(len(y2)):
            #     y2[j]=y2[j]+BS_bids['Quantity'][i]
    
    for i in range(len(BD)):
        if BD[i] * Delta_D[i][h] >0:
            k= BD_bids['Quantity'][i]
            y1 = list(map(lambda x:x+k,y1))
            # for j in range(len(y1)):
            #     y1[j]=y1[j]+BD_bids['Quantity'][i]
    # #  add Flex
    for i in range(len(FS[h])):
        if FS[h][i] == 1:
            k = FS_bids['Quantity'][i]
            y2 = list(map(lambda x:x+k,y2))
    for i in range(len(FD[h])):
        if FD[h][i] ==1:
            k = FD_bids['Quantity'][i]
            y1 = list(map(lambda x:x+k,y1))

    
    for i in range(1000):
        if x1[i]==x2[i]:
            agg_q.append(y1[i]-y2[i])
            agg_p.append(x1[i])
        elif x1[i]<x2[i]:
            temp = find_f(x2[i-1],x2[i],y2[i-1],y2[i],x1[i])
            agg_q.append(y1[i]-temp)
            agg_p.append(x1[i])
            temp = find_f(x1[i],x1[i+1],y1[i],y1[i+1],x2[i])
            agg_q.append(temp-y2[i])
            agg_p.append(x2[i])
        elif x2[i]<x1[i]:
            temp = find_f(x1[i-1],x1[i],y1[i-1],y1[i],x2[i])
            agg_q.append(temp-y2[i])
            agg_p.append(x2[i])
            temp = find_f(x2[i],x2[i+1],y2[i],y2[i+1],x1[i])
            agg_q.append(y1[i]-temp)
            agg_p.append(x1[i])
    # while agg_q[0]*agg_q[-1]>=0:
    #     raise Exception("no root")
    for i in range(len(agg_q)-1):
        if agg_q[i]*agg_q[i+1] <0:
            grad = (agg_q[i+1]-agg_q[i])/(agg_p[i+1]-agg_p[i])
            b= agg_q[i+1] - agg_p[i+1]*grad
            Res = -b/grad          
            return Res
        

$$\left[[HS][HD][FS][FD][BS][BD] \right]$$

In [ ]:

def generate_population():
    restart = True
    while restart:
        restart = False
        population = [[] for i in range(4)]
        BS,BD = HA_B()
        FS,FD = HA_FB()
        
        
        population[2]=FS
        population[3]=FD
        
        population.append(BS)
        population.append(BD)
        
        HS = [np.zeros(999).tolist() for i in range(24)]
        HD = [np.zeros(999).tolist() for i in range(24)]
        
        for h in range(24):
            price = aggregate(h,BS,BD,FS,FD)
            if price == None:
                # print('regenerate')
                restart = True
                break
            BS_INT = np.where(HS_bids['price'][h*1000:1000+h*1000]>price)[0][0]
            breakpoint_S = BS_INT + (price-HS_bids['price'][BS_INT-1+1000*h])/abs((HS_bids['price'][BS_INT+1000*h]-HS_bids['price'][BS_INT-1+1000*h]))-1
            
            BD_INT = np.where(HD_bids['price'][h*1000:1000+h*1000]>price)[0][-1]
            breakpoint_D = BD_INT + 1-(price-HD_bids['price'][BD_INT+1+1000*h])/abs((HD_bids['price'][BD_INT+1000*h]-HD_bids['price'][BD_INT+1+1000*h]))
            
            HS[h][0:int(breakpoint_S)]=np.ones(int(breakpoint_S))
            HS[h][int(breakpoint_S)]=breakpoint_S-int(breakpoint_S)
            
            HD[h][0:int(breakpoint_D)]=np.ones(int(breakpoint_D))
            HD[h][int(breakpoint_D)]=breakpoint_D-int(breakpoint_D)
            
        population[0] = HS
        population[1] = HD
        
    return population

In [ ]:
def fit(population):
    Error = False
    #objective value 
    gamma = 100000
    Hour_S = population[0]
    Hour_D = population[1]
    Block_S = population[4]
    Block_D = population[5]
    Flex_S = population[2]
    Flex_D = population[3]
    
    
    Res_HD = sum(0.5*(2*HD_bids['price'][1000*i+j]+Hour_D[i][j]*(HD_bids['price'][1000*i+j+1]-HD_bids['price'][1000*i+j]))*Hour_D[i][j]*(HD_bids['Quantity'][1000*i+j+1]-HD_bids['Quantity'][1000*i+j]) for j in range(999) for i in range(24))                
    Res_HS = sum(0.5*(2*HS_bids['price'][1000*i+j]+Hour_S[i][j]*(HS_bids['price'][1000*i+j+1]-HS_bids['price'][1000*i+j]))*Hour_S[i][j]*(HS_bids['Quantity'][1000*i+j+1]-HS_bids['Quantity'][1000*i+j]) for j in range(999) for i in range(24))  
     
    Res_BD = sum(BD_bids['Duration'][i]*BD_bids['Quantity'][i]*BD_bids['price'][i]*Block_D[i] for i in range(len(Block_D))) 
    Res_BS =  sum(BS_bids['Duration'][i]*BS_bids['Quantity'][i]*BS_bids['price'][i]*Block_S[i] for i in range(len(Block_S)))
    
    Res_FD = sum(FD_bids['Quantity'][i]*FD_bids['price'][i] *sum(Flex_D[h][i] for h in range(24)) for i in range(len(FD_bids))) 
    Res_FS = sum(FS_bids['Quantity'][i]*FS_bids['price'][i] *sum(Flex_S[h][i] for h in range(24)) for i in range(len(FS_bids)))
    
    BA = sum(Pair(1,h,HD_bids,'DQ')[0]-Pair(1,h,HD_bids,'DP')[0] for h in range(24))
    OBJ_V = Res_HD - Res_HS + Res_BD - Res_BS + Res_FD - Res_FS + BA
    price=[]
    for h in range(24):
        price.append(aggregate(h,Block_S,Block_D,Flex_S,Flex_D))
        if aggregate(h,Block_S,Block_D,Flex_S,Flex_D)==None:
            print('AGG Error')
            Error=True
            
    
    for i in range(len(Child_D)):
        if Block_D[Child_D[i]]== 1:
            if Block_D[Parent_D[i]]!=1:
                Error = True
                print('D Parent check fail')
                
    for i in range(len(Child_S)):
        if Block_S[Child_S[i]]== 1:
            if Block_S[Parent_S[i]]!=1:
                Error = True
                print('S Parent check fail',i)
                
                break
    
  
    # penalty by constarint
    Penalty = 0
    #  # penalty for constraint 6-7    
    for h in range(24):
        x = sum((HS_bids['price'][1000*h+j+1]-HS_bids['price'][1000*h+j]) * Hour_S[h][j] for j in range(len(Hour_S[h])-1))
        y = 2000+sum((HD_bids['price'][1000*h+j+1]-HD_bids['price'][1000*h+j]) * Hour_D[h][j] for j in range(len(Hour_D[h])-1))
        for i in range(len(FS_bids)):
            if x-FS_bids['price'][i] > gamma*sum(Flex_S[k][i] for k in range(24)):
                print('FS Error')
                Error = True
                break
        for i in range(len(FD_bids)):
            if FD_bids['price'][i]-x > gamma*sum(Flex_D[k][i] for k in range(24)):
                print('FD Error')
                Error = True
                break        
        # if round(x,5) == round(y,5):
        #     Error=False
        # else:
        #     Error=True
    return OBJ_V,Error,price

In [673]:
Sol_set= []
for i in tqdm(range(100)):
    Sol_set.append(generate_population())

100%|██████████| 100/100 [00:13<00:00,  7.47it/s]


In [861]:

for i in tqdm(range(100),ncols=10):
    Sol_set[i].append(fit(Sol_set[i])[0])

100%|█| 10


In [714]:
random.randint(3,10-3)

7

In [690]:
x= [1,2,3,4,5,6,7,8,9,0]
len(x)

10

In [851]:
def cross(A,B,SLD_size):
    List1 = copy.deepcopy(A)
    List2 = copy.deepcopy(B)
    for i in range(2,4):
        for h in range(24):
            begin = random.randint(SLD_size,len(List1[i][h])-SLD_size)
            temp = List1[i][h][begin:begin+SLD_size] 
            List1[i][h][begin:begin+SLD_size]=List2[i][h][begin:begin+SLD_size]
            List2[i][h][begin:begin+SLD_size] = temp
    for i in range(4,6):
        begin = random.randint(SLD_size,len(List1[i])-SLD_size)
        temp =List1[i][begin:begin+SLD_size] 
        List1[i][begin:begin+SLD_size]=List2[i][begin:begin+SLD_size]
        List2[i][begin:begin+SLD_size] = temp
    return List1,List2

def mut(pop):
    temp = copy.deepcopy(pop)
    Pm = 0.03
    for i in range(4,6):
        for j in range(len(temp[i])):
            if np.random.rand(1)<Pm:
                temp[i][j]=int(not temp[i][j])
    for i in range(2,4):
        for j in range(24):
            for k in range(len(temp[i][j])):
                    if np.random.rand(1)<Pm:
                        temp[i][j][k]=int(not temp[i][j][k])
                
        
    return temp
                
    


In [ ]:
Error= True
while tqdm(Error!=False):
    x=mut(Sol_set[0])
    
    Error=fit(x)[1]

In [862]:
sorted_id = sorted(range(len(Sol_set)), key=lambda k: Sol_set[k][6], reverse=True)
Pm= 0.05
# for i in range(10):
    
# p1 = Sol_set[sorted_id[0]]
# p2 = Sol_set[sorted_id[1]]

Error= True
while Error!=False:
    # cross
    p1 = Sol_set[sorted_id[0]]
    p2 = Sol_set[sorted_id[1]]
    
    p1,p2 = cross(p1,p2,4)
    #mut
    p1 = mut(p1)
    p2 = mut(p2)
    # mark
    Error=fit(p2)[1]
    Error=fit(p1)[1]
fit(p1)
        
        
        
    

D Parent check fail
S Parent check fail 16
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
D Parent check fail
S Parent check fail 30
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
AGG Error
S Parent check fail 9
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FS Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
D Parent check fail
D Parent check fail
D Parent check fail
S Parent check fail 9
FD Error
FS Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error
FD Error

In [ ]:
# 按列表a中元素的值进行排序，并返回元素对应索引序列

sorted_id = sorted(range(len(Sol_set)), key=lambda k: Sol_set[k][6], reverse=True)
print('元素索引序列：', sorted_id)


In [ ]:
x = list(np.arange(20))
y = list(np.zeros(20))

In [ ]:
select(x,y,4)

In [ ]:

%lprun -T lprof0 -f select 
print(open('lprof0', 'r').read())